### 📌 대분류별 정답률, 카테고리화된 정답률 feature 계산

In [ ]:
%load_ext nb_black
%load_ext lab_black

import numpy as np
import pandas as pd

import os

In [ ]:
"""
train data 또는 test data 중 feature를 추가하고 싶은 데이터를 불러오면 됩니다.!
(train과 test data를 합친 data를 불러와도 됩니다!)
"""
# TODO : 원하는 데이터의 경로를 설정해주세요
DATA_PATH = "/opt/ml/input/data/FE_total_data.csv"

raw_data = pd.read_csv(DATA_PATH, parse_dates=["Timestamp"])
raw_data = raw_data.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)

user_id = sorted(raw_data["userID"].unique())  # 유저 아이디 저장

In [ ]:
##-------------------대분류 Feature 추가--------------------------
raw_data["bigClass"] = raw_data["testId"].str[2]
raw_data.head(5)

In [ ]:
# 사용자별로 다시 대분류 별로 정답률 계산
result = (
    raw_data.groupby(["userID", "bigClass"])
    .apply(lambda x: np.mean(x["answerCode"]))
    .reset_index()
)

In [ ]:
"""
acc_by_big_category
    - key : 유저 아이디
    - value : 해당 유저의 대분류별 정답률 (인덱스가 대분류를 나타낸다. ex. index=1 -> 대분류 1)
    - default value : 기본적으로 0.5라는 값이 들어간다. 
"""
acc_by_big_category = dict()
for user in user_id:
    acc_by_big_category[user] = [0.5] * 10

for user_id, big_class, accuarcy in result.values:
    acc_by_big_category[user_id][int(big_class)] = accuarcy

In [ ]:
# 유저의 대분류별 정답률 feature 추가 (50초)
raw_data["bigClassAcc"] = raw_data.apply(
    lambda x: acc_by_big_category[x.userID][int(x.bigClass)], axis=1
)

In [ ]:
"""
acc_cate_list : 각 대분류 정답률을 카테고리로 만들어 저장하려고 한 리스트. 
ex. 원래 값 [0.04, 0.79, 0.43, 0.99] -> 곱하기 10 : [0.4, 7.9, 4.3, 9.9] -> round : [0, 8, 4, 10] -> 더하기 1 : [1, 9, 5, 11]
"""
acc_cate_list = list()
for big_class_acc in raw_data["bigClassAcc"]:
    acc = big_class_acc * 10
    acc = round(acc) + 1
    acc_cate_list.append(acc)

raw_data["bigClassAccCate"] = acc_cate_list  # 리스트로 저장

In [ ]:
raw_data = raw_data.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)
raw_data.head(5)  # 대분류, 대분류별 정답률, 대분류별 정답률 카테고리 추가 후

In [ ]:
os.makedirs("/opt/ml/input/data/FE", exist_ok=True)
raw_data.to_csv(
    "/opt/ml/input/data/FE/bigClassAcc.csv",
    columns=[
        "userID",
        "assessmentItemID",
        "testId",
        "answerCode",
        "Timestamp",
        "KnowledgeTag",
        "dataset",
        "bigClassAcc",
    ],
    index=False,
)  # 최종 결과 csv로 반환

raw_data.to_csv(
    "/opt/ml/input/data/FE/bigClassAccCate.csv",
    columns=[
        "userID",
        "assessmentItemID",
        "testId",
        "answerCode",
        "Timestamp",
        "KnowledgeTag",
        "dataset",
        "bigClassAccCate",
    ],
    index=False,
)  # 최종 결과 csv로 반환